<a href="https://colab.research.google.com/github/varunchach/transformers_demo/blob/main/Prompt_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ✅ Install necessary libraries
!pip install -q transformers datasets peft accelerate
!pip install huggingface_hub
!pip install openai==0.28

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
openai.api_key = userdata.get('OpenAI_keys')  # Replace with your key

In [ ]:
# ✅ Imports
import openai
import numpy as np
import os
import tiktoken
from google.colab import userdata
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Trainer, TrainingArguments, DataCollatorForSeq2Seq
from datasets import load_dataset
from peft import PromptTuningConfig, PromptTuningInit, get_peft_model, TaskType

Tokenization

In [ ]:
# Text to analyze
text = "Hello ChatGPT myself Varun"

# 1. Tokenization using tiktoken (same tokenizer used by OpenAI)
encoding = tiktoken.encoding_for_model("text-embedding-3-small")
tokens = encoding.encode(text)

print("Original Text:", text)
print("Number of Tokens:", len(tokens))
print("Token IDs:", tokens)



Original Text: Hello ChatGPT myself Varun
Number of Tokens: 7
Token IDs: [9906, 13149, 38, 2898, 7182, 8909, 359]


Embeddings

In [ ]:
# 2. Get Embeddings from OpenAI
response = openai.embeddings.create(
    model="text-embedding-3-small",
    input=text
)

embedding = response.data[0].embedding

print("\nLength of Embedding Vector:", len(embedding))
print("First 5 Dimensions of Embedding Vector:", embedding[:5])


Length of Embedding Vector: 1536
First 5 Dimensions of Embedding Vector: [-0.023479679599404335, 0.0009394785156473517, 0.022358132526278496, -0.012548228725790977, -0.0032444780226796865]


In [ ]:
# Define cosine similarity
def cosine_similarity(a, b):
    a = np.array(a)
    b = np.array(b)
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# Words from same and different semantic groups
group_fruit = ["mango", "banana"]
group_profession = ["doctor", "engineer"]

# Combine all words
all_words = group_fruit + group_profession

# Get embeddings
response = openai.embeddings.create(
    model="text-embedding-3-large",
    input=all_words
)

# Store embeddings in a dictionary
embeddings = {word: res.embedding for word, res in zip(all_words, response.data)}

# Compare similarities
similar_same_group = cosine_similarity(embeddings["mango"], embeddings["banana"])
similar_diff_group = cosine_similarity(embeddings["mango"], embeddings["doctor"])

# Display results
print(f"Cosine Similarity (Same Group - 'mango' vs 'banana'): {similar_same_group:.4f}")
print(f"Cosine Similarity (Different Group - 'mango' vs 'doctor'): {similar_diff_group:.4f}")


Cosine Similarity (Same Group - 'mango' vs 'banana'): 0.5327
Cosine Similarity (Different Group - 'mango' vs 'doctor'): 0.2639



## 🔄 **Prompt Drift in LLMs**

### 🧠 What is Prompt Drift?

* When the **same prompt** yields **different outputs** over time
* Caused by **model updates**, **randomness**, or **context changes**

---

### ⚙️ Causes of Prompt Drift

* 🧬 **Model updates** or LLM migrations
* 📥 Changes in **prompt injection** or RAG data
* 🎲 LLM’s **creative randomness**
* 🔁 **Prompt chaining** leading to cascading inaccuracies
* 🧭 Off-topic **model tangents or biases**

---

### 🚨 Impact of Prompt Drift

* ❌ **Inconsistent responses** across sessions
* 📉 **Reduced reliability** in production systems
* 🧪 **Hard to test/debug** for developers

---

### 🛠️ Mitigation Strategies

* 🧪 Regular **prompt testing & benchmarking**
* 📋 Use of **comprehensive templates**
* 🧠 Stronger **in-context learning (ICL)**
* 📈 Continuous **monitoring & logging**


In [ ]:
def call_model(prompt, model):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0.7
    )
    return response.choices[0].message.content.strip()

In [ ]:
prompt = "Explain me results for Ind vs Pak finals 2007."

# Compare across versions (if available to you)
models = ["gpt-3.5-turbo", "gpt-4-turbo"]
for model in models:
    print(f"\n🧠 Model: {model}")
    print(call_model(prompt, model))



🧠 Model: gpt-3.5-turbo
In the 2007 ICC World Twenty20 final between India and Pakistan, India emerged victorious by defeating Pakistan by 5 runs. India batted first and scored 157/5 in their 20 overs, with Gautam Gambhir top-scoring with 75 runs. In response, Pakistan managed to score 152/7 in their 20 overs, falling short of the target by 5 runs. Irfan Pathan was the pick of the Indian bowlers, taking 3 wickets for 16 runs. This victory marked India's first-ever win in a World T20 final and further added to the intense rivalry between the two cricketing nations.

🧠 Model: gpt-4-turbo
The India vs. Pakistan final in the 2007 ICC World Twenty20 held in South Africa is one of the most memorable matches in the history of cricket, particularly in the T20 format. The match took place on September 24, 2007, at the Wanderers Stadium in Johannesburg.

### Match Summary:
- **Toss**: India won the toss and chose to bat first.
- **India's Innings**: India posted a total of 157/5 in their 20 over


### **🧠 Memory Handling in LLMs**

---

#### 📌 Why Memory Matters:

* Enables **context retention** in conversations
* Supports **factual consistency** and task tracking
* Combines short-term and long-term memory strategies

---

#### 1️⃣ **Working Memory (Short-Term)**

* Like human short-term memory
* Limited by the **context window** (e.g., 4k, 8k, 32k tokens)
* Helps LLM stay coherent in a single conversation

---

#### 2️⃣ **Long-Term Memory**

* Stores info **across sessions**
* Techniques:

  * External DBs for persistent storage
  * Agent-based context management
  * **Memory tuning** (injecting fixed facts)

---

#### 3️⃣ **Memory Management Techniques**

* **Context Window Optimization**
* **Agents** to track and recall info
* **PagedAttention (vLLM)** for better GPU usage

---

#### 4️⃣ **Key Applications**

* 🤖 **Conversational Memory** — Chatbots, Customer Support
* 🧭 **Agent Memory** — Autonomous agents tracking goals



In [ ]:
# 💾 Simulated long-term memory store
long_term_memory = {}

# 🧠 Function to query OpenAI with short-term + optional memory
def query_llm(user_input, conversation_history, memory_summary=None):
    messages = []

    # Inject memory summary if available
    if memory_summary:
        messages.append({"role": "system", "content": f"Here’s what you already know about the user: {memory_summary}"})

    # Add past conversation history (short-term memory)
    messages.extend(conversation_history)

    # Add latest user input
    messages.append({"role": "user", "content": user_input})

    # Get response
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0.7
    )

    reply = response.choices[0].message.content
    return reply


In [ ]:
# Initial Conversation

conversation_history = []

# Step 1: User shares favorite food
user_input1 = "My favorite food is sushi."
response1 = query_llm(user_input1, conversation_history)
conversation_history.append({"role": "user", "content": user_input1})
conversation_history.append({"role": "assistant", "content": response1})

print(response1)


Sushi is a traditional Japanese dish made with vinegared rice, seaweed, and various ingredients such as raw or cooked fish, vegetables, and sauces. It is known for its fresh and delicate flavors, and I love the combination of different textures and flavors in each bite. Sushi is also a healthy option as it is low in calories and high in protein and omega-3 fatty acids. Whether I'm enjoying simple nigiri sushi or elaborate sushi rolls, I always find it to be a delicious and satisfying meal.


In [ ]:
long_term_memory["favorite_food"] = "sushi"

In [ ]:
# Clear short-term memory (simulate new session)
conversation_history = []

# Inject memory summary
memory_summary = f"The user's favorite food is {long_term_memory.get('favorite_food')}."

# New unrelated prompt
user_input2 = "Can you suggest a dinner plan for me?"

response2 = query_llm(user_input2, conversation_history, memory_summary)
conversation_history.append({"role": "user", "content": user_input2})
conversation_history.append({"role": "assistant", "content": response2})

print(response2)


Sure! How about a sushi dinner plan? You can start with some miso soup or seaweed salad as appetizers. For the main course, you can enjoy a variety of sushi rolls such as California rolls, spicy tuna rolls, and salmon avocado rolls. You can also add some nigiri sushi like salmon, tuna, and eel. To complement the meal, you can have some edamame and green tea. For dessert, you can finish off with some mochi ice cream or green tea ice cream. Enjoy your sushi dinner!


## 🤯 Hallucination in LLMs

### 🔍 What is Hallucination?

LLM hallucination occurs when a language model generates **confident-sounding but factually incorrect or entirely made-up information**.

---

### 🚨 Why It Happens:

* 🔄 **No real understanding** — LLMs generate based on patterns, not truth
* 🧠 **Trained to predict** next words, not verify facts
* 📉 **Missing context** or vague prompts
* 🏗️ **Lack of access to real-time knowledge** (if not connected to tools or RAG)

---

### 📌 Real-World Examples:

#### **1. Inventing People:**

**Prompt:** *"Tell me about Dr. Rajeev Saxena, the Nobel-winning data scientist from India."*
**LLM Output:** *"Dr. Rajeev Saxena won the Nobel Prize in 2021 for his work on neural networks."*
👉 **Fact:** No such person exists.

---

#### **2. Fabricated References:**

**Prompt:** *"Give me citations for AI fairness research."*
**LLM Output:** *“See Smith et al., 2020, Journal of Ethical AI, Vol. 4(3).”*
👉 **Fact:** The journal and paper may not exist at all.

---

#### **3. Failing at Math:**

**Prompt:** *"What’s 847 multiplied by 63?"*
**LLM Output:** *"The answer is 53,421."*
👉 **Fact:** It doesn’t actually calculate; it tries to "guess" the answer.

---

#### **4. Hallucinated Product Features:**

**Prompt:** *"What features does the iPhone 20 have?"*
**LLM Output:** *"The iPhone 20 includes a quantum chip and foldable screen."*
👉 **Fact:** That product doesn't exist yet.

---

### 🛡️ How to Reduce Hallucination:

* ✅ Use **RAG (Retrieval-Augmented Generation)**
* ✅ Integrate with **external knowledge sources**
* ✅ Employ **fact-checking agents**
* ✅ Apply **prompt tuning** and **instruction tuning**

---


Incorrect Math

In [ ]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    temperature=0.9,  # Increase randomness
    messages=[
        {"role": "user", "content": "Learn addition and substraction what is 111 minus 11 after adding 1 to it"}
    ]
)

print("Likely Hallucination:\n", response['choices'][0]['message']['content'])


Likely Hallucination:
 111 - 11 = 100

When you add 1 to 111, it becomes 112. So, 112 - 11 = 100


Fabricated Facts

In [ ]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    temperature=0.9,  # Increase randomness
    messages=[
        {"role": "user", "content": "Can you tell me about the final which was played between India and West Indies 2007 WC"}
    ]
)

print("Likely Hallucination:\n", response['choices'][0]['message']['content'])


Likely Hallucination:
 The final of the 2007 ICC Cricket World Cup was played between India and West Indies on April 28, 2007 at the Kensington Oval in Bridgetown, Barbados. 

India won the toss and elected to bat first. They posted a total of 157 runs for the loss of 5 wickets in their allotted 20 overs. Yuvraj Singh was the star performer for India, scoring 70 runs off just 30 balls.

In reply, West Indies struggled to chase down the target due to some excellent bowling by the Indian team. They were restricted to just 98 runs for the loss of 4 wickets in their 20 overs, handing India a comfortable victory by 59 runs.

Yuvraj Singh was named the Player of the Match for his brilliant innings with the bat. This victory marked India's first ICC T20 World Cup win and cemented their reputation as a force to be reckoned with in the shortest format of the game.


### 🔧 Prompt Tuning in LLMs

* 🎯 **What is Prompt Tuning?**
  A lightweight fine-tuning method where *learnable prompt vectors* are optimized while keeping the LLM frozen.

* 🧠 **LLM Stays Frozen**
  Only the prompt embeddings are updated; model weights are not modified.

* ⚡ **Efficient & Fast**
  Requires fewer resources and is faster than full fine-tuning.

* 🧩 **Task-Specific Prompts**
  Learns prompts tailored for downstream tasks like classification, summarization, etc.

* 💡 **Soft vs Hard Prompts**

  * *Soft prompts:* Learnable vectors
  * *Hard prompts:* Manually written text

* 📚 **Great for Low-Resource Scenarios**
  Works well when labeled data is limited.

* 🔍 **Used With Adapter Frameworks**
  Commonly used with PEFT libraries (e.g., Hugging Face + LoRA, Prompt-Tuning APIs)



In [ ]:
# ✅ Load a small model for fast experimentation
model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# ✅ STEP 3: Load IMDb dataset & sample subset
dataset = load_dataset("imdb", split="train").select(range(2000))
print(dataset[:5])

{'text': ['I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far b

In [ ]:
# ✅ STEP 4: Convert to prompt-response format
def format_prompt(example):
    return {
        "input": f"Classify the sentiment: {example['text'][:300]}",
        "output": "positive" if example["label"] == 1 else "negative"
    }

formatted_dataset = dataset.map(format_prompt)



Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
# ✅ STEP 5: Tokenize
model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(batch):
    inputs = tokenizer(batch["input"], padding="max_length", truncation=True, max_length=256)
    targets = tokenizer(batch["output"], padding="max_length", truncation=True, max_length=10)
    inputs["labels"] = targets["input_ids"]
    return inputs

tokenized_dataset = formatted_dataset.map(tokenize_function, remove_columns=formatted_dataset.column_names)



Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
# ✅ STEP 6: Load base model and apply PEFT config

# Load the base sequence-to-sequence language model from Hugging Face Hub
# e.g., model_name = "google/flan-t5-base" or "t5-small"
base_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Define configuration for PEFT (Prompt Tuning in this case)
peft_config = PromptTuningConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,               # Specify this is a sequence-to-sequence task
    prompt_tuning_init=PromptTuningInit.TEXT,      # Initialize soft prompts from a text string
    prompt_tuning_init_text="Classify the sentiment:",  # Initialization text for virtual prompt tokens
    num_virtual_tokens=10,                         # Number of learnable virtual tokens added to the input
    tokenizer_name_or_path=model_name              # Use the same tokenizer as the base model
)

# Apply the PEFT wrapper to the base model using the defined prompt tuning config
# This returns a model that adds and trains soft prompt tokens, keeping base model frozen
model = get_peft_model(base_model, peft_config)

In [ ]:
# ✅ STEP 7: Training

# Define training configurations using HuggingFace's TrainingArguments
training_args = TrainingArguments(
    output_dir="./results",                  # Folder to save model checkpoints or logs
    per_device_train_batch_size=4,           # Batch size per device (GPU/CPU) during training
    num_train_epochs=3,                      # Number of passes through the full dataset
    logging_steps=10,                        # Log training loss every 10 steps
    save_strategy="no",                      # Don't save model checkpoints during training
    report_to="none"                         # Don't report to tools like WandB or TensorBoard
)

# Create a Trainer object that handles training and evaluation
trainer = Trainer(
    model=model,                             # The pre-trained or custom model you want to train
    args=training_args,                      # Training arguments defined above
    train_dataset=tokenized_dataset,         # The tokenized dataset to train on
    tokenizer=tokenizer,                     # Tokenizer (helps with padding, decoding, etc.)
    data_collator=DataCollatorForSeq2Seq(tokenizer, model)  # Handles dynamic padding of inputs
)

# Start training
trainer.train()


<ipython-input-24-d7db62f3a056>:11: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
10,48.387500
20,47.883600
30,46.726600
40,47.439400
50,47.046500
60,47.464100
70,48.135500
80,46.587400
90,47.745900
100,46.764500


TrainOutput(global_step=1500, training_loss=47.43219694010417, metrics={'train_runtime': 125.0524, 'train_samples_per_second': 47.98, 'train_steps_per_second': 11.995, 'total_flos': 557671514112000.0, 'train_loss': 47.43219694010417, 'epoch': 3.0})

In [ ]:
# ✅ Manually curated strongly polar examples
test_examples = [
    {
        "text": "Not sure but not a good movie guys ,First half is really great and fun to watch but in second half they ruined everything, I was sure that it was a good direction but ruined by actors somehow",
        "expected": "negative"
    },
    {
        "text": "Difficult to predict I was going to rate this negatively but overall If I compare every bad with good then it was a good experience in all.",
        "expected": "positive"
    }
]

def predict_sentiment(model, text):
    prompt = f"Classify the sentiment: {text[:300]}"
    inputs = tokenizer(prompt, return_tensors="pt").to("cpu")
    output = model.generate(**inputs, max_new_tokens=5)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# ✅ Compare base vs fine-tuned output
for example in test_examples:
    review = example["text"]
    print("\n📝 Review Snippet:")
    print(review[:250].replace("\n", " ") + "...")

    print("🎯 Expected:", example['expected'])
    print("📦 Original Model Prediction:", predict_sentiment(original_model, review))
    print("✅ Fine-Tuned Model Prediction:", predict_sentiment(tuned_model, review))
    print("-" * 80)



📝 Review Snippet:
Not sure but not a good movie guys ,First half is really great and fun to watch but in second half they ruined everything, I was sure that it was a good direction but ruined by actors somehow...
🎯 Expected: negative
📦 Original Model Prediction: negative
✅ Fine-Tuned Model Prediction: negative
--------------------------------------------------------------------------------

📝 Review Snippet:
Difficult to predict I was going to rate this negatively but overall If I compare every bad with good then it was a good experience in all....
🎯 Expected: positive
📦 Original Model Prediction: negative
✅ Fine-Tuned Model Prediction: positive
--------------------------------------------------------------------------------
